In [1]:
import numpy as np
import pandas as pd

# Question 1

What's the version of NumPy that you installed?

In [2]:
np.__version__

'1.21.5'

# Question 2

How many records are in the dataset?

In [3]:
df = pd.read_csv('data.csv.zip')
print(df.shape)

(11914, 16)


In [4]:
df.head()

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350
3,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916,29450
4,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916,34500


# Question 3

Who are the most popular car manufacturers (top-3) according to the dataset?

In [5]:
df.Make.value_counts()[:5]

Chevrolet     1123
Ford           881
Volkswagen     809
Toyota         746
Dodge          626
Name: Make, dtype: int64

Answer: Chevrolet, Ford, Volkswagen

# Question 4

What's the number of unique Audi car models in the dataset?

In [6]:
df.query('Make == "Audi"')['Model'].nunique()

34

# Question 5

How many columns in the dataset have missing values?

In [7]:
df.isna().any().sum()

5

# Question 6

Find the median value of "Engine Cylinders" column in the dataset.

In [8]:
df['Engine Cylinders'].median()

6.0

Next, calculate the most frequent value of the same "Engine Cylinders"

In [9]:
df['Engine Cylinders'].mode()

0    4.0
Name: Engine Cylinders, dtype: float64

In [10]:
df['Engine Cylinders'].value_counts(dropna=False)

4.0     4752
6.0     4489
8.0     2031
12.0     230
5.0      225
10.0      68
0.0       56
NaN       30
3.0       30
16.0       3
Name: Engine Cylinders, dtype: int64

Use the fillna method to fill the missing values in "Engine Cylinders" with the most frequent value from the previous step.

In [11]:
df['Engine Cylinders'] = df['Engine Cylinders'].fillna(df['Engine Cylinders'].mode()[0])

In [12]:
df['Engine Cylinders'].value_counts(dropna=False)

4.0     4782
6.0     4489
8.0     2031
12.0     230
5.0      225
10.0      68
0.0       56
3.0       30
16.0       3
Name: Engine Cylinders, dtype: int64

In [13]:
df['Engine Cylinders'].median()

6.0

Has it changed?

Answer: No

# Question 7

Select all the "Lotus" cars from the dataset.

Select only columns "Engine HP", "Engine Cylinders"

In [14]:
df_lotus = df.query('Make == "Lotus"')[['Engine HP','Engine Cylinders']].copy()
print(df_lotus.shape)
df_lotus.head()

(29, 2)


,Engine HP,Engine Cylinders
3912,189.0,4.0
3913,218.0,4.0
3914,189.0,4.0
3915,189.0,4.0
3916,218.0,4.0


Now drop all duplicated rows using drop_duplicates method (you should get a dataframe with 9 rows).

In [15]:
df_lotus = df_lotus.drop_duplicates()
df_lotus.shape

(9, 2)

Get the underlying NumPy array. Let's call it X

In [16]:
X = df_lotus.values
X

array([[189.,   4.],
       [218.,   4.],
       [217.,   4.],
       [350.,   8.],
       [400.,   6.],
       [276.,   6.],
       [345.,   6.],
       [257.,   4.],
       [240.,   4.]])

Compute matrix-matrix multiplication between the transpose of X and X. To get the transpose, use X.T. Let's call the result XTX

In [17]:
XTX = X.T.dot(X)
XTX

array([[7.31684e+05, 1.34100e+04],
       [1.34100e+04, 2.52000e+02]])

Invert XTX

In [18]:
XTX_inv = np.linalg.inv(XTX)
XTX_inv

array([[ 5.53084235e-05, -2.94319825e-03],
       [-2.94319825e-03,  1.60588447e-01]])

Create an array y with values [1100, 800, 750, 850, 1300, 1000, 1000, 1300, 800].

In [19]:
y = [1100, 800, 750, 850, 1300, 1000, 1000, 1300, 800]

Multiply the inverse of XTX with the transpose of X, and then multiply the result by y. Call the result w

In [20]:
w = XTX_inv.dot(X.T).dot(y)
w[0], w[1]

(4.594944810094579, -63.56432501336555)

What's the value of the first element of w

Answer: 4.5949

In [21]:
df_lotus['y'] = y 
df_lotus['pred'] = (df_lotus['Engine HP']*w[0] + df_lotus['Engine Cylinders']*w[1]).astype('int')
df_lotus

,Engine HP,Engine Cylinders,y,pred
3912,189.0,4.0,1100,614
3913,218.0,4.0,800,747
3918,217.0,4.0,750,742
4216,350.0,8.0,850,1099
4257,400.0,6.0,1300,1456
4259,276.0,6.0,1000,886
4262,345.0,6.0,1000,1203
4292,257.0,4.0,1300,926
4293,240.0,4.0,800,848


In [22]:
# RMSE
sum((df_lotus['y'] - df_lotus['pred'])*2)/len(df_lotus)

84.22222222222223